<a href="https://colab.research.google.com/github/anniepeacock/DANSAR/blob/devel/burn_severity/GetData_UAVSAR_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Downloading Available UAVSAR Data (for Time Series & Regeneration) & Making GIS-Ready Products**

*This notebook runs through downloading and pre-processing UAVSAR Data to be used for fire severity analysis over the Verdugo Mountains in Los Angeles*

- Inputs: GRD, INC, and Annotation files
- Outputs: Masked, cropped, incidence-angle correction, speckle filtered, geotiffs

----------------

Table of Contents:
0. [Running the Notebook](#s1)
1. [Download UAVSAR Data](#s2)
2. [Generate a HDR File](#s3)
3. [Read in data and plot](#s4)
4. [Crop and export to geotiff](#s5)

<a name="s1"></a>
## Running the Notebook
- This Notebook has both "text" and "code" cells. The text cells have text descriptions about running the notebooks and data interpretation.
- Code cells are a light gray and a "play" button appears in the upper left corner when your mouse is hovered over the cell.
- To run the content in the code cells, **select the play button** in the upper left corner of each code cell or **press shift-enter**.

### Python Packages:
This routine uses the following python libraries. Some are already included in the Google Colab environment and others are installed in the cell below before imported. Downloading new python packages to this environment may take a few minutes to complete.

In [20]:
!pip install rasterio --quiet        # Install python package "rasterio" into Google Colab environment
!pip install earthpy --quiet         # For plot legends
## Using ASF Search
!pip install asf_search --quiet

In [21]:
# Scripts and functions needed from GitHub repos
# To build HDR files
!git clone https://bitbucket.org/nathanmthomas/bucket-of-rs-and-gis-scripts/src/master/BuildUAVSARhdr.py

# To appy Enhanced Lee Speckle Filter
!git clone https://github.com/NaiaraSPinto/VegMapper.git

# To generate geotiffs and apply incidence angle masking
!wget https://raw.githubusercontent.com/anniepeacock/DANSAR/devel/utils/functions.py

fatal: destination path 'BuildUAVSARhdr.py' already exists and is not an empty directory.
fatal: destination path 'VegMapper' already exists and is not an empty directory.
--2024-05-15 23:42:47--  https://raw.githubusercontent.com/anniepeacock/DANSAR/devel/utils/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5783 (5.6K) [text/plain]
Saving to: ‘functions.py.1’

functions.py.1      100%[===================>]   5.65K  --.-KB/s    in 0s      

2024-05-15 23:42:47 (42.4 MB/s) - ‘functions.py.1’ saved [5783/5783]



In [22]:
import rasterio as rio               # Reading and writing raster formats
import numpy as np                   # Numeric and mathematic functions
import matplotlib.pyplot as plt      # Plotting, making graphs
import getpass                       # Discreet way to enter username and password
import os
from osgeo import gdal
from skimage import filters          # For otsu thresholding
import math
import geopandas as gpd
from matplotlib.patches import Patch          # Creates and plots visualization i.e. legends or figures - (Section 3, 4)
from rasterio.plot import plotting_extent     # Creates an object to plot raster and vector data together - (Section 3. HV Change Detection)
import requests
from osgeo import gdal
from rasterio.plot import show_hist
from rasterio.mask import mask as rio_mask
from shapely.geometry import mapping
from rasterio.plot import show
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import earthpy.plot as ep
from skimage import filters
from scipy.ndimage.filters import generic_filter
from scipy.stats import mode


# Read in functions from functions.py script
from functions import * #download_files, process_grd_files, process_inc_files, mask_and_save, convert_db

# Read in enhanced lee filter from VegMapper
import sys
path_to_module = '/content/VegMapper/vegmapper/core/'
sys.path.append(path_to_module)
from filter import enhanced_lee

<ipython-input-22-4f7ddbcd74a1>:22: DeprecationWarning: Please use `generic_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import generic_filter


<a name="s2"></a>
# 1. Download UAVSAR and NISAR Simulated Data

An Earthdata account is required to download the UAVSAR or NISAR Simulated data stored at ASF to this notebook environment. Create an account here, if needed: https://urs.earthdata.nasa.gov//users/new

In [23]:
username = getpass.getpass(prompt='Earthdata username:')
password = getpass.getpass(prompt='Earthdata password:')

Earthdata username:··········
Earthdata password:··········


Now, download the UAVSAR data and its corresponding annotation file using wget and your Earthdata username and password. The annotation file (.ann) is required to generate a HDR (.hdr) file. This hdr file stores the georeferencing information of an associated raw raster file and is needed to make the UAVSAR GRD files (.grd) GIS-readable.


Browse for UAVSAR data of interest here: https://uavsar.jpl.nasa.gov/cgi-bin/data.pl

* To download here, replace the links below with the new data of interest

In [32]:
import asf_search as asf
session = asf.ASFSession() # Downloading requires session

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

Auth failed: Username or password is incorrect


In [33]:
## How do you get HV polarizations from ASF??
help(asf.constants.POLARIZATION)

Help on module asf_search.constants.POLARIZATION in asf_search.constants:

NAME
    asf_search.constants.POLARIZATION

DATA
    DUAL_HH = ''
    DUAL_HV = ''
    DUAL_VH = ''
    DUAL_VV = ''
    FULL = 'full'
    HH = 'HH'
    HH_3SCAN = ''
    HH_4SCAN = ''
    HH_5SCAN = ''
    HH_HV = 'HH+HV'
    HH_HV_VH_VV = 'HH+HV+VH+VV'
    HH_VV = 'HH+VV'
    QUAD = 'quadrature'
    UNKNOWN = 'UNKNOWN'
    VV = 'VV'
    VV_VH = 'VV+VH'

FILE
    /usr/local/lib/python3.10/dist-packages/asf_search/constants/POLARIZATION.py




In [30]:
# can generate polygon using ASF Vertex AOI tool
wkt = 'POLYGON((-118.3582 34.2487,-118.2604 34.1655,-118.2241 34.1865,-118.3224 34.2681,-118.3582 34.2487))'
results = asf.geo_search(platform=[asf.PLATFORM.UAVSAR], intersectsWith=wkt, processingLevel=asf.PRODUCT_TYPE.AMPLITUDE_GRD, flightLine='08525')
print(results)

{
  "features": [
    {
      "geometry": {
        "coordinates": [
          [
            [
              -119.278706,
              34.236551
            ],
            [
              -119.262257,
              34.036094
            ],
            [
              -116.055173,
              34.239516
            ],
            [
              -116.064811,
              34.431078
            ],
            [
              -119.278706,
              34.236551
            ]
          ]
        ],
        "type": "Polygon"
      },
      "properties": {
        "beamModeType": "RPI",
        "browse": [
          "https://datapool.asf.alaska.edu/BROWSE/UA/SanAnd_08525_18076-003_20026-016_0708d_s01_L090HH_01.cor.png",
          "https://datapool.asf.alaska.edu/BROWSE/UA/SanAnd_08525_18076-003_20026-016_0708d_s01_L090HH_01.hgt.png"
        ],
        "bytes": 1727326957,
        "centerLat": 34.245539482218,
        "centerLon": -117.679610845501,
        "fileID": "UA_SanAnd_08525_18076

In [36]:
urls = ['https://uavsar.asf.alaska.edu/UA_SanAnd_08525_23008_011_230712_L090_CX_01/SanAnd_08525_23008_011_230712_L090HVHV_CX_01.grd']
asf.download_urls(urls=urls,  path = './', session=session)

ASFAuthenticationError: HTTP 401: HTTP Basic: Access denied.


In [ ]:
granule_list = [
    'UA_SanAnd_08525_14158_003_141023_L090HVHV_CX_01-PROJECTED',
    'UA_SanAnd_26526_14092_007_140624_L090HVHV_CX_01-PROJECTED',
    'UA_SanAnd_08525_17122_003_171102_L090HVHV_CX_01-PROJECTED',
    'UA_SanAnd_26526_17122_004_171102_L090HVHV_CX_01-PROJECTED',
    'UA_SanAnd_26526_17122_004_171102_L090_CX_01-INC',
    'UA_SanAnd_08525_17122_003_171102_L090_CX_01-INC',
    'UA_SanAnd_26526_14092_007_140624_L090_CX_01-METADATA',
    'UA_SanAnd_08525_14158_003_141023_L090_CX_01-METADATA',
    'UA_SanAnd_26526_17122_004_171102_L090_CX_01-METADATA',
    'UA_SanAnd_08525_17122_003_171102_L090_CX_01-METADATA',
]
results = asf.granule_search(granule_list=granule_list)

print(f'{len(results)} results found')

6 results found


In [ ]:
results.download(path='./', session=user_pass_session, processes=10)

/usr/local/lib/python3.10/dist-packages/asf_search/download/download.py:68: UserWarning: File already exists, skipping download: ./SanAnd_26526_17122_004_171102_L090_CX_01.inc
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.10/dist-packages/asf_search/download/download.py:68: UserWarning: File already exists, skipping download: ./SanAnd_08525_17122_003_171102_L090_CX_01.ann
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.10/dist-packages/asf_search/download/download.py:68: UserWarning: File already exists, skipping download: ./SanAnd_26526_14092_007_140624_L090_CX_01.ann
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.10/dist-packages/asf_search/download/download.py:68: UserWarning: File already exists, skipping download: ./SanAnd_26526_17122_004_171102_L090_CX_01.ann
  warnings.warn(f'File 

<a name="s3"></a>
# 2. Generate a HDR file

Generate HDR files for each of the GRDs, so they can be GIS-compatible and converted to geotiffs.

In [ ]:
!python3 "/content/BuildUAVSARhdr.py/BuildUAVSARhdr.py" -h

UAVSAR.py is written by Nathan Thomas (nmthomas28@gmail.com, @DrNASApants)  formerly of the Aberystwyth University Earth Observation and Ecosystems Dynamics Laboratory (@AU_EarthObs) as part of a visiting research program at NASA JPL. Ammended to process UAVSAR InSAR pair data by Yang Zheng (April 2015).
Use '-h' for help and required input parameters

usage: BuildUAVSARhdr.py [-h] [-i INPUT] [-r UAVSAR] [-p POLARIZATION]

options:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        Specify the input UAVSAR ann file
  -r UAVSAR, --uavsar UAVSAR
                        Specify the input UAVSAR radar file
  -p POLARIZATION, --polarization POLARIZATION
                        Specify the input UAVSAR polarization in UPPERCASE (i.e HHHV, HHHH, VVVV,
                        AMP1, AMP2, COR, UNW, HGT, INT)


In [ ]:
def run_hdr_script(directory, polarization):
    file_data = []

    # Find GRD and annotation files
    for filename in os.listdir(directory):
        if filename.endswith('.grd'):
            grd_file = filename
            annotation_file = filename.replace(f'_L090{polarization}_', '_L090_').replace('.grd', '.ann')
            file_data.append((grd_file, annotation_file))
        elif filename.endswith('.inc'):
            grd_file = filename
            annotation_file = filename.replace(f'_L090{polarization}_', '_L090_').replace('.inc', '.ann')
            file_data.append((grd_file, annotation_file))

    # Run HDR script for each file
    for grd_file, annotation_file in file_data:
        !python3 "/content/BuildUAVSARhdr.py/BuildUAVSARhdr.py" -i {annotation_file} -r {grd_file} -p {polarization}

# Run HDR script for HVHV polarization
run_hdr_script('./', 'HVHV')

# Run HDR script for HHHH polarization
run_hdr_script('./', 'HHHH')

# Run HDR script for incidence angle files (assuming .inc files)
run_hdr_script('./', 'HVHV')  # You can put whichever polarization, using HVHV as a placeholder here

UAVSAR.py is written by Nathan Thomas (nmthomas28@gmail.com, @DrNASApants)  formerly of the Aberystwyth University Earth Observation and Ecosystems Dynamics Laboratory (@AU_EarthObs) as part of a visiting research program at NASA JPL. Ammended to process UAVSAR InSAR pair data by Yang Zheng (April 2015).
Use '-h' for help and required input parameters

UPPER LEFT LAT =  34.4433
UPPER LEFT LONG =  -119.2884
SAMPLES = 4136
Lines = 32329
PIXEL SIZE =  0.0001
DATATYPE =  4
SanAnd_08525_14158_003_141023_L090HVHV_CG_138A_02.grd.hdr
Writing output HDR file...
Output HDR file = SanAnd_08525_14158_003_141023_L090HVHV_CG_138A_02.grd.hdr

Thank you for using UAVSAR.py

UAVSAR.py is written by Nathan Thomas (nmthomas28@gmail.com, @DrNASApants)  formerly of the Aberystwyth University Earth Observation and Ecosystems Dynamics Laboratory (@AU_EarthObs) as part of a visiting research program at NASA JPL. Ammended to process UAVSAR InSAR pair data by Yang Zheng (April 2015).
Use '-h' for help and requi

<a name="s5"></a>
# 4. Crop the data and export subset to geotiff

In [ ]:
# Define input directory and output directory
input_directory = './' # Directory containing GRD and INC files
output_directory = './'  # Directory to save processed files

# Define extent and projection details
upper_left_x = 373624.5024220281047747
upper_left_y = 3780585.1359808142296970
lower_right_x = 387664.5024220281047747
lower_right_y = 3791835.1359808142296970
window = (upper_left_x, upper_left_y, lower_right_x, lower_right_y)

projection = 'EPSG:26911' # reprojecting here to be in meters to match the Landsat data that will be reprojected in meters.

In [ ]:
# Process GRD files
process_grd_files(input_directory, output_directory, window, projection)

In [ ]:
# Process incidence angle files
process_inc_files(input_directory, output_directory)

# 5. Incidence Angle Corrections/Masking

Converting sigma0 to gamma0 should remove most of the range/incidence angle dependency.

gamma0 = sigma0 / cos(incidence angle)

In [ ]:
mask_and_save(input_directory, output_directory)

In [ ]:
## Pre-Fire
# Open the two input rasters using rasterio
with rio.open('SanAnd_08525_14158_003_141023_L090HVHV_02_masked.tif') as src1:
    arr1 = src1.read(1, masked=True)
    meta1 = src1.meta

with rio.open('SanAnd_26526_14092_007_140624_L090HVHV_02_masked.tif') as src2:
    arr2 = src2.read(1, masked=True)
    meta2 = src2.meta

In [ ]:
cmap = 'Greys_r'
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,10))

vmin_HV=0
vmax_HV=0.1

# 2014 SanAnd_08525
plot = ax1.imshow(arr1.squeeze(),vmin=vmin_HV, vmax=vmax_HV, cmap=cmap)
ax1.set_title('Incidence Angle Masked - Heading 1')
fig.colorbar(plot, ax=ax1, shrink=0.5)

# 2014 SanAnd_26526
plot = ax2.imshow(arr2.squeeze(), vmin=vmin_HV, vmax=vmax_HV,cmap=cmap)
ax2.set_title('Incidence Angle Masked - Heading 2')
fig.colorbar(plot, ax=ax2, shrink=0.5)

In [ ]:
# Merge the two arrays using numpy's maximum function to take the maximum value at each pixel
merged_arr_prefire = np.nanmax((arr1, arr2), axis=0)

## Post-Fire
# Open the two input rasters using rasterio
with rio.open('SanAnd_26526_17122_004_171102_L090HVHV_02_masked.tif') as src1:
    arr1 = src1.read(1, masked=True)
    meta1 = src1.meta

with rio.open('SanAnd_08525_17122_003_171102_L090HVHV_02_masked.tif') as src2:
    arr2 = src2.read(1, masked=True)
    meta2 = src2.meta

# Merge the two arrays using numpy's maximum function to take the maximum value at each pixel
merged_arr_postfire = np.nanmax((arr1, arr2), axis=0)

In [ ]:
fig, axs = plt.subplots(figsize=(5,5))
plt.imshow(merged_arr_prefire, vmin=0, vmax=0.1, cmap="Greys_r")
plt.title('Pre-Fire merged opposing headings')

In [ ]:
fig, axs = plt.subplots(figsize=(5,5))
plt.imshow(merged_arr_postfire, vmin=0, vmax=0.1, cmap="Greys_r")
plt.title('Post-Fire merged opposing headings')

In [ ]:
plt.figure(figsize=(3, 3))
plt.hist(merged_arr_postfire.flatten(), range = [-0.25, 1.5], bins=200, alpha=0.7, color='blue')
plt.hist(merged_arr_prefire.flatten(), range = [-0.25, 1.5], bins=200, alpha=0.7, color='red')
plt.xlabel('Pixel Values')
plt.ylabel('Frequency')
plt.title('Histogram of NISAR Sim')
plt.show()

# Lee Filter

In [ ]:
lee_merged_arr_prefire = enhanced_lee(merged_arr_prefire, 5, 1)
lee_merged_arr_postfire = enhanced_lee(merged_arr_postfire, 5, 1)

# Export
- HV_log
- fire severity product
- send to UAVSAR staging area

In [ ]:
# Open the reference raster
with rio.open(reference_path) as src:
    # Get metadata from the reference raster
    meta = src.meta

    # Update metadata to match the NumPy array
    meta.update({
        'dtype': HV_log.dtype,
        'count': 1  # Assuming there's only one band
    })

    # Save HV Log Ratio
    with rio.open("HV_log_uavsar.tif", 'w', **meta) as dst:
        dst.write(HV_log, 1)  # Assuming there's only one band (band index 1)

    # Save HV Log Ratio Classification
    with rio.open("HV_log_uavsar_classification.tif", 'w', **meta) as dst:
        dst.write(hv_class, 1)  # Assuming there's only one band (band index 1)

    # Filtered Classification
    with rio.open("HV_log_uavsar_classification_filtered.tif", 'w', **meta) as dst:
        dst.write(filtered_image, 1)  # Assuming there's only one band (band index 1)

or export a data stack

1. HV Pre-Fire (Merged opposite headings and Lee Filtered)
2. HV Post-Fire (Merged opposite headings and Lee Filtered)
3. HV Log
4. Fire Severity Classification

Could add:
- incidence angles
- slope
- aspect

In [ ]:
# Open the reference raster
with rio.open(reference_raster) as src:
    # Get metadata from the reference raster
    meta = src.meta

    # Update metadata to handle multiple bands
    meta.update({
        'dtype': 'float32',  # Assuming all bands are of float32 dtype
        'count': 4,          # Number of bands
        'driver': 'GTiff'
    })

    # Add band descriptions to the metadata
    meta['descriptions'] = ('HV Pre-Fire', 'HV Post-Fire','HV Log Ratio', 'HV Log Ratio Classification')

    # Create a new stack with multiple bands
    with rio.open("uavsar_stack.tif", 'w', **meta) as dst:
        dst.write(merged_arr_prefire, 1)
        dst.write(merged_arr_postfire, 2)
        dst.write(HV_log, 3)
        dst.write(hv_class, 4)
